# Fetch subgraph data


In [1]:
import os
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [3]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)

def clean_df(df, decimal_cols):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [4]:
# queries
position_query = gql("""
query futuresPositions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
    }  
}
""")

trade_query = gql("""
query futuresTrades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")


### Run a query

In [5]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, position_cols)
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,openTimestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x0,0x8feafd5e0fcefdd2624906c0f913d563306aae17,0x4d415449430000000000000000000000000000000000...,77.199529739466510105,0,1.336,1.336,1.29958209269590481,1.29958209269590481,0,...,1675952475,1676027925,0,0,False,False,5418.135835493916478991,15359.399172591107662112,24028.921415094259736673,0
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,0x4d415449430000000000000000000000000000000000...,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,1675952535,1675952960,7.861001237100389934,0,False,False,71.390620323035134147,0,0,3861.411655626170581108
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x2,0x5bbefaccc1a8e1ecb0d470f941668562eeb46ff0,0x4d415449430000000000000000000000000000000000...,48,-152.5638,1.310855380043199703,1.310855380043199703,1.310855380043199703,NaN,0,...,1675960102,NaN,2.239986893635801653,0,False,True,47.760013106364198347,0,0,199.989078029834710848
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x3,0x5f9908aa2af7653cc50f9d93416b8c96e18b957f,0x4d415449430000000000000000000000000000000000...,48,-192.4515,1.297930826701847969,1.297930826701847969,1.297930826701847969,NaN,0,...,1675960767,NaN,2.299746481394012833,0,False,True,47.700253518605987167,0,0,249.788734495010694406
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x4,0xee6b71b0b9e8a2e7a30f3f966a41c8b414d5e2cc,0x4d415449430000000000000000000000000000000000...,48,-154.464,1.293132711810875521,1.293132711810875521,1.293132711810875521,NaN,0,...,1675961288,NaN,2.239690941436586091,0,False,True,47.760309058563413909,0,0,199.742451197155076475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1881,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0x0,0x0c4ea5f0219fbc4decf9536463524953cd75f0cb,0x58414700000000000000000000000000000000000000...,99.348053173628019585,0,22.307998,22.307998,22.0345,22.0345,0,...,1675898432,1676057525,0,0,False,False,0,322.337473307573198351,3035.773205108572818559,0
1882,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0x1,0x0c4ea5f0219fbc4decf9536463524953cd75f0cb,0x58414700000000000000000000000000000000000000...,99.348053173628019585,0,22.316497516160874295,22.316497516160874295,22.308497517051278839,22.308497517051278839,0.890404444875241552,...,1675899039,1675899209,0.745062614332699782,0,False,False,99.493395000960673566,0,0,4966.787294069706803125
1883,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0x2,0xafa306dbb13beb550511ffbd304d97461e71603c,0x58414700000000000000000000000000000000000000...,2382.822300402661836278,0,22.4179761517786778,22.4179761517786778,22.38047619167109905,22.38047619167109905,39.892378812526048335,...,1675920280,1675921611,11.150469547872419007,0,False,False,2413.564191118385578535,0,0,47656.499526211951368497
1884,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0x3,0x9e5a57470a2a136a3d4de39e1996e05944c4b1fb,0x58414700000000000000000000000000000000000000...,248,25.5723,22.376250572212177875,22.376250572212177875,22.376250572212177875,NaN,0,...,1675953728,NaN,2.114442438501556295,0,False,True,247.885557561498443705,0,0,572.212192507781476272


In [6]:
trade_params = {
    'last_id': ''
}

trade_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, trade_cols)
df_trade


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed
0,0x0003d5401b4c7a8a1feb9e8ea750336e6df42b5cd3ff...,0x50d36f27872f637ec9ecb4eedff300df44a608ab,1673640747,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1420.509945340577467893,83.803545657223511638,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x26b,3.128026947594952567,0,-1.5882,0,True
1,0x00232eaefdd653149d9ae21764a1f412d548a5b6dd4d...,0xa110d783a5b6b0b2a2c867094364e8f9752d5b93,1675964339,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1613.322643212031584915,19212.662872131648312194,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x5f5,331.155086968312641252,0,-204.0231,0,True
2,0x0024ea62644b2054a4e2acc5e37b6291fa2973117a33...,0xadc6461555c94217d14351ded5f5583634c08191,1673742652,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,Market,1542.283461883403867843,398.000000000000080497,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x2b1,2.698996058295956768,0,-3.5,-3.5,False
3,0x00268abb5859495ac5ed1519f64ae84b1f7c50eef1d7...,0xc9c67603d3107ad53d1816d3096b5b36b52cdedc,1675342588,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,Market,1675.63495283188297317,43.758239544222270206,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x4d1,0.000004212759417786,0,0.000005028254406685,-4.14103137E-10,False
4,0x002a6dfbe14b59d9b22c3489c22d21266eedf5a3a316...,0xa2b3b5ef8ceb806c86e4c281c047a07d7e947645,1673621772,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1411.810788757105774211,49259.444074234867972493,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x24e,319.263048697128246744,0,-300,-300.758714704386296336,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4583,0xffbd5dc7a59a78954c991953cd5cd4d5432e5f182a64...,0xf68d2bfcecd7895bba05a7451dd09a1749026454,1674181460,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1560.374513038230526637,422.309324946709399037,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x360,8.702120608401807758,0,4.2952,0,True
4584,0xffc382f11d1ee63c101ca5f077ed51b10f56ce904bdf...,0x4ee0b41fb8460f7fb7761ee83c7de8b02780a232,1675684359,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1631.815684315594942266,98.900867550947062148,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x5ac,2.252197114010975198,0,0.3091,0.3091,False
4585,0xffd64e62c995b0cde44238bfa23d9bf789f5a1f0e102...,0xf0753b791d9afa14e4f0963c2f9b603282d593fd,1673542151,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1385.055469509376174366,108.221843840186870326,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x23d,2.364182547749186308,0,1.969859803856687971,1.969859803856687971,False
4586,0xffe545b86e9a47ba061ddbe7f20d4729169f347009c1...,0x0c8aa570a1dfeee5258f3c13e2e967da24bbb505,1673262644,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1322.79752629640388717,52.755917631435047066,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x190,1.137240243353251903,0,0.2075,0,True
